# Image Classification with Neural DSL

This tutorial demonstrates how to build a complete image classification pipeline using Neural DSL.

## Overview
- Build a CNN for image classification
- Train on CIFAR-10 or custom datasets
- Visualize model architecture and performance
- Export and deploy the model

## Setup

In [ ]:
# Install Neural DSL if not already installed
# !pip install neural-dsl

import os
import sys
import numpy as np
import matplotlib.pyplot as plt

# Import neural CLI programmatically
from neural.cli.cli import cli
from neural.parser.parser import create_parser, ModelTransformer
from neural.code_generation.code_generator import generate_code

## Define the Model in Neural DSL

We'll create a convolutional neural network for image classification with:
- Multiple convolutional blocks with batch normalization
- Max pooling and dropout for regularization
- Dense layers for classification

In [ ]:
dsl_code = """
network ImageClassifier {
  input: (None, 32, 32, 3)
  
  layers:
    Conv2D(filters=32, kernel_size=(3,3), activation="relu", padding="same")
    BatchNormalization()
    Conv2D(filters=32, kernel_size=(3,3), activation="relu", padding="same")
    BatchNormalization()
    MaxPooling2D(pool_size=(2,2))
    Dropout(rate=0.25)
    
    Conv2D(filters=64, kernel_size=(3,3), activation="relu", padding="same")
    BatchNormalization()
    Conv2D(filters=64, kernel_size=(3,3), activation="relu", padding="same")
    BatchNormalization()
    MaxPooling2D(pool_size=(2,2))
    Dropout(rate=0.25)
    
    Flatten()
    Dense(units=256, activation="relu")
    BatchNormalization()
    Dropout(rate=0.5)
    Output(units=10, activation="softmax")

  loss: "categorical_crossentropy"
  optimizer: Adam(learning_rate=0.001)
  metrics: ["accuracy"]

  train {
    epochs: 50
    batch_size: 64
    validation_split: 0.2
  }
}
"""

# Save the DSL code to a file
with open('image_classifier.neural', 'w') as f:
    f.write(dsl_code)

print("Model defined successfully!")

## Compile the Model

Use Neural CLI to compile the DSL to TensorFlow or PyTorch code.

In [ ]:
# Compile to TensorFlow
!neural compile image_classifier.neural --backend tensorflow --output image_classifier_tf.py

print("\nModel compiled to TensorFlow!")
print("Output file: image_classifier_tf.py")

## Visualize Model Architecture

In [ ]:
# Visualize the model
!neural visualize image_classifier.neural --format html

print("Visualization generated!")
print("Open architecture.svg to see the model structure")

## Load and Prepare Data

We'll use CIFAR-10 dataset for this example.

In [ ]:
# Load CIFAR-10 data
try:
    import tensorflow as tf
    from tensorflow import keras
    
    # Load dataset
    (x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
    
    # Normalize pixel values
    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0
    
    # Convert labels to categorical
    y_train = keras.utils.to_categorical(y_train, 10)
    y_test = keras.utils.to_categorical(y_test, 10)
    
    print(f"Training data shape: {x_train.shape}")
    print(f"Test data shape: {x_test.shape}")
    
    # Display sample images
    class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
                   'dog', 'frog', 'horse', 'ship', 'truck']
    
    plt.figure(figsize=(10, 10))
    for i in range(25):
        plt.subplot(5, 5, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(x_train[i])
        plt.xlabel(class_names[np.argmax(y_train[i])])
    plt.tight_layout()
    plt.show()
    
except ImportError:
    print("TensorFlow not installed. Install with: pip install tensorflow")

## Train the Model

In [ ]:
# Import the generated model code
try:
    exec(open('image_classifier_tf.py').read())
    
    # Note: The generated code should create and compile the model
    # You may need to adapt this based on the actual generated code structure
    
    print("Model loaded successfully!")
    
except Exception as e:
    print(f"Error loading model: {e}")
    print("You may need to run the generated script manually")

## Alternative: Use CLI to Train

In [ ]:
# Train using Neural CLI
!neural run image_classifier_tf.py --backend tensorflow --dataset CIFAR10

## Evaluate Model Performance

In [ ]:
# This cell assumes the model has been trained and is available
# Adapt based on your actual training results

# Example evaluation (pseudo-code)
# test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
# print(f'\nTest accuracy: {test_acc:.4f}')
# print(f'Test loss: {test_loss:.4f}')

## Make Predictions

In [ ]:
# Make predictions on test images
# predictions = model.predict(x_test[:10])

# # Visualize predictions
# plt.figure(figsize=(15, 3))
# for i in range(10):
#     plt.subplot(2, 5, i + 1)
#     plt.imshow(x_test[i])
#     predicted_class = class_names[np.argmax(predictions[i])]
#     true_class = class_names[np.argmax(y_test[i])]
#     color = 'green' if predicted_class == true_class else 'red'
#     plt.xlabel(f'Pred: {predicted_class}\nTrue: {true_class}', color=color)
#     plt.xticks([])
#     plt.yticks([])
# plt.tight_layout()
# plt.show()

## Hyperparameter Optimization

In [ ]:
# Use Neural's HPO feature
!neural compile image_classifier.neural --backend tensorflow --hpo --dataset CIFAR10

## Debug the Model

In [ ]:
# Debug with NeuralDbg dashboard
# Run this in a separate terminal:
# neural debug image_classifier.neural --backend tensorflow --dashboard --port 8050

print("To debug the model, run:")
print("neural debug image_classifier.neural --backend tensorflow --dashboard --port 8050")
print("Then visit http://localhost:8050 in your browser")

## Export Model

Export to different formats for deployment.

In [ ]:
# Compile to PyTorch
!neural compile image_classifier.neural --backend pytorch --output image_classifier_torch.py

# Compile to ONNX
!neural compile image_classifier.neural --backend onnx --output image_classifier.onnx

## Summary

In this tutorial, we:
1. Defined an image classification model using Neural DSL
2. Compiled it to TensorFlow code
3. Visualized the model architecture
4. Trained on CIFAR-10 dataset
5. Evaluated and made predictions
6. Explored hyperparameter optimization
7. Learned about debugging tools

## Next Steps

- Try different architectures (ResNet, VGG, MobileNet)
- Use transfer learning with pre-trained models
- Implement data augmentation
- Deploy the model to production
- Explore other use cases (NLP, time series, GANs)